In [28]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import keras
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LogisticRegression

In [29]:
dataset = pd.read_csv("data/iplmatches.csv")

In [30]:
dataset.head()

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,N,NaN,MR Benson,SL Shastri
2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,N,NaN,SJ Davis,DJ Harper
4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,N,NaN,BF Bowden,K Hariharan


In [31]:
dataset.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816 entries, 0 to 815
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               816 non-null    int64  
 1   city             803 non-null    object 
 2   date             816 non-null    object 
 3   player_of_match  812 non-null    object 
 4   venue            816 non-null    object 
 5   neutral_venue    816 non-null    int64  
 6   team1            816 non-null    object 
 7   team2            816 non-null    object 
 8   toss_winner      816 non-null    object 
 9   toss_decision    816 non-null    object 
 10  winner           812 non-null    object 
 11  result           812 non-null    object 
 12  result_margin    799 non-null    float64
 13  eliminator       812 non-null    object 
 14  method           19 non-null     object 
 15  umpire1          816 non-null    object 
 16  umpire2          816 non-null    object 
dtypes: float64(1), i

In [32]:

conditions = [dataset["venue"] == "Rajiv Gandhi International Stadium, Uppal",dataset["venue"] == "Maharashtra Cricket Association Stadium",
              dataset["venue"] == "Saurashtra Cricket Association Stadium", dataset["venue"] == "Holkar Cricket Stadium",
              dataset["venue"] == "M Chinnaswamy Stadium",dataset["venue"] == "Wankhede Stadium",
              dataset["venue"] == "Eden Gardens",dataset["venue"] == "Feroz Shah Kotla",
              dataset["venue"] == "Punjab Cricket Association IS Bindra Stadium, Mohali",dataset["venue"] == "Green Park",
              dataset["venue"] == "Punjab Cricket Association Stadium, Mohali",dataset["venue"] == "Dr DY Patil Sports Academy",
              dataset["venue"] == "Sawai Mansingh Stadium", dataset["venue"] == "MA Chidambaram Stadium, Chepauk", 
              dataset["venue"] == "Newlands", dataset["venue"] == "St George's Park" , 
              dataset["venue"] == "Kingsmead", dataset["venue"] == "SuperSport Park",
              dataset["venue"] == "Buffalo Park", dataset["venue"] == "New Wanderers Stadium",
              dataset["venue"] == "De Beers Diamond Oval", dataset["venue"] == "OUTsurance Oval", 
              dataset["venue"] == "Brabourne Stadium",dataset["venue"] == "Sardar Patel Stadium", 
              dataset["venue"] == "Barabati Stadium", dataset["venue"] == "Vidarbha Cricket Association Stadium, Jamtha",
              dataset["venue"] == "Himachal Pradesh Cricket Association Stadium",dataset["venue"] == "Nehru Stadium",
              dataset["venue"] == "Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium",dataset["venue"] == "Subrata Roy Sahara Stadium",
              dataset["venue"] == "Shaheed Veer Narayan Singh International Stadium",dataset["venue"] == "JSCA International Stadium Complex",
              dataset["venue"] == "Sheikh Zayed Stadium",dataset["venue"] == "Sharjah Cricket Stadium",
              dataset["venue"] == "Dubai International Cricket Stadium",dataset["venue"] == "M. A. Chidambaram Stadium",
              dataset["venue"] == "Feroz Shah Kotla Ground",dataset["venue"] == "M. Chinnaswamy Stadium",
              dataset["venue"] == "Rajiv Gandhi Intl. Cricket Stadium" ,dataset["venue"] == "IS Bindra Stadium",dataset["venue"] == "ACA-VDCA Stadium"]



In [33]:
values = ['Hyderabad', 'Mumbai', 'Rajkot',"Indore","Bengaluru","Mumbai","Kolkata","Delhi","Mohali","Kanpur","Mohali","Pune","Jaipur","Chennai","Cape Town","Port Elizabeth","Durban",
          "Centurion",'Eastern Cape','Johannesburg','Northern Cape','Bloemfontein','Mumbai','Ahmedabad','Cuttack','Jamtha','Dharamshala','Chennai','Visakhapatnam','Pune','Raipur','Ranchi',
          'Abu Dhabi','Sharjah','Dubai','Chennai','Delhi','Bengaluru','Hyderabad','Mohali','Visakhapatnam']


#Removing records having null values in "winner" column



dataset['city'] = np.where(dataset['city'].isnull(),
                              np.select(conditions, values),
                              dataset['city'])

# #Removing records having null values in "winner" column
# dataset=dataset[dataset["winner"].notna()]

In [34]:
for t in dataset['team1'].unique():
    print(t)

Royal Challengers Bangalore
Kings XI Punjab
Delhi Daredevils
Mumbai Indians
Kolkata Knight Riders
Rajasthan Royals
Deccan Chargers
Chennai Super Kings
Kochi Tuskers Kerala
Pune Warriors
Sunrisers Hyderabad
Gujarat Lions
Rising Pune Supergiants
Rising Pune Supergiant
Delhi Capitals


In [35]:
dataset=dataset[dataset["winner"].notna()]
print(dataset)
encoder = LabelEncoder()


id        city        date player_of_match  \
0     335982   Bangalore  2008-04-18     BB McCullum   
1     335983  Chandigarh  2008-04-19      MEK Hussey   
2     335984       Delhi  2008-04-19     MF Maharoof   
3     335985      Mumbai  2008-04-20      MV Boucher   
4     335986     Kolkata  2008-04-20       DJ Hussey   
..       ...         ...         ...             ...   
811  1216547       Dubai  2020-09-28  AB de Villiers   
812  1237177       Dubai  2020-11-05       JJ Bumrah   
813  1237178   Abu Dhabi  2020-11-06   KS Williamson   
814  1237180   Abu Dhabi  2020-11-08      MP Stoinis   
815  1237181       Dubai  2020-11-10        TA Boult   

                                          venue  neutral_venue  \
0                         M Chinnaswamy Stadium              0   
1    Punjab Cricket Association Stadium, Mohali              0   
2                              Feroz Shah Kotla              0   
3                              Wankhede Stadium              0   
4      

In [36]:

dataset['team1'] = encoder.fit_transform(dataset['team1'])
dataset['team2'] = encoder.fit_transform(dataset['team2'])
# dataset['winner'] = encoder.fit_transform(dataset['winner'].astpye(str))
dataset['toss_winner'] = encoder.fit_transform(dataset['toss_winner'])


dataset['venue'] = encoder.fit_transform(dataset['venue'])

dataset['winner'] = encoder.fit_transform(dataset['winner'].astype(str))


In [37]:
dataset.loc[dataset["winner"]==dataset["team1"],"team1_win"]=1

dataset.loc[dataset["winner"]!=dataset["team1"],"team1_win"]=0
dataset.loc[dataset["toss_winner"]==dataset["team1"],"team1_toss_win"]=1
dataset.loc[dataset["toss_winner"]!=dataset["team1"],"team1_toss_win"]=0

dataset["team1_bat"]=0
dataset.loc[(dataset["team1_toss_win"]==1) & (dataset["toss_decision"]=="bat"),"team1_bat"]=1

print(dataset)


id        city        date player_of_match  venue  neutral_venue  \
0     335982   Bangalore  2008-04-18     BB McCullum     14              0   
1     335983  Chandigarh  2008-04-19      MEK Hussey     23              0   
2     335984       Delhi  2008-04-19     MF Maharoof      8              0   
3     335985      Mumbai  2008-04-20      MV Boucher     35              0   
4     335986     Kolkata  2008-04-20       DJ Hussey      7              0   
..       ...         ...         ...             ...    ...            ...   
811  1216547       Dubai  2020-09-28  AB de Villiers      6              0   
812  1237177       Dubai  2020-11-05       JJ Bumrah      6              0   
813  1237178   Abu Dhabi  2020-11-06   KS Williamson     30              0   
814  1237180   Abu Dhabi  2020-11-08      MP Stoinis     30              0   
815  1237181       Dubai  2020-11-10        TA Boult      6              0   

     team1  team2  toss_winner toss_decision  winner   result  result_mar

In [38]:
dataset.loc[dataset["toss_winner"]==dataset["team1"],"team1_toss_win"]=1
dataset.loc[dataset["toss_winner"]!=dataset["team1"]]

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2,team1_win,team1_toss_win,team1_bat
1,335983,Chandigarh,2008-04-19,MEK Hussey,23,0,5,0,0,bat,0,runs,33.0,N,NaN,MR Benson,SL Shastri,0.0,0.0,0
2,335984,Delhi,2008-04-19,MF Maharoof,8,0,3,10,10,bat,3,wickets,9.0,N,NaN,Aleem Dar,GA Pratapkumar,1.0,0.0,0
4,335986,Kolkata,2008-04-20,DJ Hussey,7,0,7,1,1,bat,7,wickets,5.0,N,NaN,BF Bowden,K Hariharan,1.0,0.0,0
5,335987,Jaipur,2008-04-21,SR Watson,27,0,10,5,5,bat,10,wickets,6.0,N,NaN,Aleem Dar,RB Tiffin,1.0,0.0,0
7,335989,Chennai,2008-04-23,ML Hayden,16,0,0,8,8,field,0,runs,6.0,N,NaN,DJ Harper,GA Pratapkumar,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801,1216537,Abu Dhabi,2020-10-30,BA Stokes,30,0,5,10,10,field,10,wickets,7.0,N,NaN,CB Gaffaney,S Ravi,0.0,0.0,0
803,1216539,Dubai,2020-09-25,PP Shaw,6,0,2,0,0,field,2,runs,44.0,N,NaN,KN Ananthapadmanabhan,RK Illingworth,1.0,0.0,0
811,1216547,Dubai,2020-09-28,AB de Villiers,6,0,13,8,8,field,13,tie,NaN,Y,NaN,Nitin Menon,PR Reiffel,1.0,0.0,0
812,1237177,Dubai,2020-11-05,JJ Bumrah,6,0,8,2,2,field,8,runs,57.0,N,NaN,CB Gaffaney,Nitin Menon,1.0,0.0,0


In [39]:
prediction_df = dataset[['team1','team2','team1_toss_win','team1_bat','team1_win','venue']]

correlated_featues = set()
correlation_matrix = prediction_df.drop('team1_win',axis=1).corr()

for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i,j]) > 0.8:
            column = correlation_matrix.columns[i]
            correlated_features.add(column)



In [40]:
prediction_df.drop(columns= correlated_featues)
X = prediction_df.drop('team1_win',axis=1)
target = prediction_df['team1_win']
target = target.astype(int)





In [41]:
reg = LogisticRegression(solver = 'lbfgs')
rfe = rfe.fit(X,target.values.ravel())

NameError: name 'LogisticRegression' is not defined

In [42]:
X_train,X_test,y_train,y_test = tts(X,target,test_size=0.2,random_state=0)

In [43]:
print(X_train)

team1  team2  team1_toss_win  team1_bat  venue
37       5     10             0.0          0     23
172      0      1             1.0          1      4
48       5      1             1.0          0     23
320      3      0             1.0          0     16
627      3     11             1.0          1      8
..     ...    ...             ...        ...    ...
767      8      5             0.0          0     30
192      6      0             1.0          0     18
632      8     11             1.0          0     35
562      5     14             1.0          1     22
687      3      0             0.0          0      8

[649 rows x 5 columns]


In [44]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()



X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

print(X_train)
print(X_test)

[[-0.42111856  0.64791865 -0.95627103 -0.53875639  0.45760002]
 [-1.54676082 -1.41898077  1.04572864  1.85612647 -1.4295792 ]
 [-0.42111856 -1.41898077  1.04572864 -0.53875639  0.45760002]
 ...
 [ 0.25426681  0.87757414  1.04572864 -0.53875639  1.64950269]
 [-0.42111856  1.56654062  1.04572864  1.85612647  0.3582748 ]
 [-0.87137546 -1.64863627 -0.95627103 -0.53875639 -1.03227831]]
[[-0.8771527  -0.6098245   1.0061539  -0.48461168 -0.95480836]
 [-1.54630662  0.5994382  -0.99388373 -0.48461168 -0.14315901]
 [-0.8771527   0.84129074  1.0061539  -0.48461168  1.7845082 ]
 [-0.8771527   0.35758566 -0.99388373 -0.48461168 -0.95480836]
 [-0.8771527  -1.57723466 -0.99388373 -0.48461168 -0.95480836]
 [ 1.3533604  -1.57723466 -0.99388373 -0.48461168 -0.34607135]
 [-0.43105008  0.11573312  1.0061539  -0.48461168 -1.05626453]
 [ 0.68420647 -0.85167704 -0.99388373 -0.48461168 -1.66500154]
 [-1.54630662  0.35758566 -0.99388373 -0.48461168 -0.14315901]
 [-0.43105008 -1.57723466  1.0061539   2.06350782

In [55]:
lgr = LogisticRegression()
lgr.fit(X_train,y_train)
y_pred = lgr.predict(X_test)

NameError: name 'LogisticRegression' is not defined

In [56]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test,y_pred)
print(cm)
score = lgr.score(X_test,y_test)
print(score)


[[38 29]
 [52 44]]


NameError: name 'lgr' is not defined

In [47]:
from sklearn.ensemble import RandomForestClassifier
reg = RandomForestClassifier(n_estimators=1000)
reg.fit(X_train,y_train)
y_pred = reg.predict(X_test)

confusion_matrix(y_pred,y_test)
score = reg.score(X_test,y_test)

print(score)

0.5030674846625767


In [48]:
#training model using artificial neural net
import keras
import tensorflow as tf

In [49]:
ann = tf.keras.models.Sequential()

In [50]:
print(ann)

In [51]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()


# from sklearn.preprocessing import OneHotEncoder
# ct = ColumnTransformer(transformers=[('encoder')], remainder='passthrough')

# X = np.array(ct.fit_transform(X))
# target = np.array(ct.fit_transform(target))
# X_train = sc.fit_transform(X_train)
# X_test = sc.fit_transform(X_test)

print(X_train)
print(X_test)

[[-0.42111856  0.64791865 -0.95627103 -0.53875639  0.45760002]
 [-1.54676082 -1.41898077  1.04572864  1.85612647 -1.4295792 ]
 [-0.42111856 -1.41898077  1.04572864 -0.53875639  0.45760002]
 ...
 [ 0.25426681  0.87757414  1.04572864 -0.53875639  1.64950269]
 [-0.42111856  1.56654062  1.04572864  1.85612647  0.3582748 ]
 [-0.87137546 -1.64863627 -0.95627103 -0.53875639 -1.03227831]]
[[-0.8771527  -0.6098245   1.0061539  -0.48461168 -0.95480836]
 [-1.54630662  0.5994382  -0.99388373 -0.48461168 -0.14315901]
 [-0.8771527   0.84129074  1.0061539  -0.48461168  1.7845082 ]
 [-0.8771527   0.35758566 -0.99388373 -0.48461168 -0.95480836]
 [-0.8771527  -1.57723466 -0.99388373 -0.48461168 -0.95480836]
 [ 1.3533604  -1.57723466 -0.99388373 -0.48461168 -0.34607135]
 [-0.43105008  0.11573312  1.0061539  -0.48461168 -1.05626453]
 [ 0.68420647 -0.85167704 -0.99388373 -0.48461168 -1.66500154]
 [-1.54630662  0.35758566 -0.99388373 -0.48461168 -0.14315901]
 [-0.43105008 -1.57723466  1.0061539   2.06350782

In [52]:
#adding layers in Ann
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='sigmoid'))

In [53]:
#compiling my ann
ann.compile(optimizer='adam')

In [54]:
ann.fit(X_train,y_train,batch_size=32,epochs = 100)

Epoch 1/100


ValueError: in user code:

    /Users/suryanshsoni/Desktop/ML/MLwork/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /Users/suryanshsoni/Desktop/ML/MLwork/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/suryanshsoni/Desktop/ML/MLwork/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/suryanshsoni/Desktop/ML/MLwork/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/suryanshsoni/Desktop/ML/MLwork/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **k